In [2]:
import random
import numpy as np
import torch

import mmcv
import mmrotate
import mmdet
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
# Check mmcv installation
print(mmcv.__version__) #noqa E1101
# Check MMRotate installation
print(mmrotate.__version__)
print(mmrotate.__file__)
# Check MMDetection installation
print(mmdet.__version__)

print(get_compiling_cuda_version())
print(get_compiler_version())

2.0.0
1.0.0rc1
/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/06.Gits/01.SAMRS_MTP/MTP/MMRotate_1_x/mmrotate/mmrotate/__init__.py
3.1.0
11.1
GCC 7.3


In [3]:
import mmengine
print(mmengine.__version__)

0.10.3


In [4]:
from mmengine.config import Config
from mmengine.runner import Runner

from mmdet.utils import register_all_modules as register_all_modules_mmdet
from mmdet.apis import inference_detector, init_detector

from mmrotate.registry import VISUALIZERS
from mmrotate.utils import register_all_modules

In [5]:
from mmrotate.registry import MODELS
from mmrotate.testing import get_detector_cfg
#from torchinfo import summary

In [6]:
from mmrotate.registry import DATASETS
from mmrotate.datasets.dota import DOTADataset

@DATASETS.register_module()
class ShipDataset(DOTADataset):
    """SAR ship dataset for detection."""
    
    METAINFO = {
            'classes':('ship',),
                # palette is a list of color tuples, which is used for visualization.
            'palette': [(165, 42, 42),]
        }

In [7]:
cfg = Config.fromfile('/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/06.Gits/01.SAMRS_MTP/MTP/MMRotate_1_x/fine_tune/oriented_rcnn_rvsa_l_1024_mae_mtp_dota10_adjust.py')

# Modify dataset type and path
cfg.data_root = '/mnt/hdd/eric/.tmp_ipy/00.Data/ShipRS_dataset/ShipRSImageNet_V1/DOTA_Format_MMrotate_1x/'
cfg.dataset_type = 'ShipDataset'

# modify num classes of the model in box head
cfg['model']['roi_head']['bbox_head']['num_classes']=1
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = "/mnt/hdd/eric/.tmp_ipy/00.Weights/MTP/diorr-rvsa-l-mae-mtp-epoch_12.pth"
cfg.checkpoint = cfg.load_from
cfg['model']['backbone']['pretrained'] = None

# Set up working dir to save files and logs.
cfg.work_dir = "/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/06.Gits/01.SAMRS_MTP/MTP/MMRotate_1_x/work_dir"

cfg.optim_wrapper.optimizer.lr = 0.001

cfg.train_cfg.val_interval = 1
# Change the evaluation metric since we use customized dataset.
cfg.val_evaluator.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.val_evaluator.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.default_hooks.checkpoint.interval = 1
# cfg.max_epochs=36
cfg.train_cfg.max_epochs=12

# Set seed thus the results are more reproducible
cfg.seed = 0
random.seed(cfg.seed)
np.random.seed(cfg.seed)
torch.cuda.manual_seed(cfg.seed)
torch.cuda.manual_seed_all(cfg.seed)

cfg.gpu_ids = range(1)
cfg.device='cuda:0'


# modify pipeline mainly for resize scale (512,512)
cfg.train_pipeline = [
    dict(type='mmdet.LoadImageFromFile', backend_args=None),
    dict(type='mmdet.LoadAnnotations', with_bbox=True, box_type='qbox'),
    dict(type='ConvertBoxType', box_type_mapping=dict(gt_bboxes='rbox')),
    dict(type='mmdet.Resize', scale=(800, 800), keep_ratio=True),
    dict(
        type='mmdet.RandomFlip',
        prob=0.75,
        direction=['horizontal', 'vertical', 'diagonal']),
    dict(
        type='RandomRotate',
        prob=0.5,
        angle_range=180,
        rect_obj_labels=[9, 11]),
    dict(
        type='mmdet.Pad', size=(800, 800),
        pad_val=dict(img=(114, 114, 114))),
    dict(type='mmdet.PackDetInputs')
]
cfg.val_pipeline = [
    dict(type='mmdet.LoadImageFromFile', backend_args=None),
    dict(type='mmdet.Resize', scale=(800, 800), keep_ratio=True),
    dict(type='mmdet.LoadAnnotations', with_bbox=True, box_type='qbox'),
    dict(type='ConvertBoxType', box_type_mapping=dict(gt_bboxes='rbox')),
    dict(
        type='mmdet.Pad', size=(800, 800),
        pad_val=dict(img=(114, 114, 114))),
    dict(
        type='mmdet.PackDetInputs',
        meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
                   'scale_factor'))
]
cfg.test_pipeline = [
    dict(type='mmdet.LoadImageFromFile', backend_args=None),
    dict(type='mmdet.Resize', scale=(800, 800), keep_ratio=True),
    dict(
        type='mmdet.Pad', size=(800, 800),
        pad_val=dict(img=(114, 114, 114))),
    dict(
        type='mmdet.PackDetInputs',
        meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
                   'scale_factor'))
]


cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.ann_file = 'val'
cfg.val_dataloader.dataset.data_prefix=dict(img_path='images_png', img='images_png')
# cfg.val_dataloader.dataset.img_prefix = 'images'
cfg.val_dataloader.dataset.data_root = '/mnt/hdd/eric/.tmp_ipy/00.Data/ShipRS_dataset/ShipRSImageNet_V1/DOTA_Format_MMrotate_1x/'

cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.ann_file = 'train'
cfg.train_dataloader.dataset.data_prefix=dict(img_path='images_png', img='images_png')
# cfg.train_dataloader.dataset.img_prefix = 'images'
cfg.train_dataloader.dataset.data_root = '/mnt/hdd/eric/.tmp_ipy/00.Data/ShipRS_dataset/ShipRSImageNet_V1/DOTA_Format_MMrotate_1x/'

cfg.test_dataloader.dataset.type = cfg.dataset_type
cfg.test_dataloader.dataset.data_prefix=dict(img_path='images_png', img='images_png')
cfg.test_dataloader.dataset.ann_file = 'val'
# cfg.test_dataloader.dataset.img_prefix = 'images'
cfg.test_dataloader.dataset.data_root = '/mnt/hdd/eric/.tmp_ipy/00.Data/ShipRS_dataset/ShipRSImageNet_V1/DOTA_Format_MMrotate_1x/'

cfg.train_dataloader.dataset.pipeline=cfg.train_pipeline
cfg.val_dataloader.dataset.pipeline=cfg.val_pipeline
cfg.test_dataloader.dataset.pipeline=cfg.test_pipeline

cfg.val_evaluator = dict(type='DOTAMetric', metric='mAP')
cfg.test_evaluator = cfg.val_evaluator
# We can also use tensorboard to log the training process
# cfg.vis_backends=[dict(type='TensorboardVisBackend')]
# cfg.visualizer = dict(type='mmrotate.RotLocalVisualizer',vis_backends=vis_backends)

# We can initialize the logger for training and have a look
# at the final config used for training


#----- wrapper
del cfg['optim_wrapper']['constructor']

#-----
cfg.dump('./tiny_cfg.py')
print(f'Config:\n{cfg.pretty_text}')

Config:
angle_version = 'le90'
backend_args = None
checkpoint = '/mnt/hdd/eric/.tmp_ipy/00.Weights/MTP/diorr-rvsa-l-mae-mtp-epoch_12.pth'
data_root = '/mnt/hdd/eric/.tmp_ipy/00.Data/ShipRS_dataset/ShipRSImageNet_V1/DOTA_Format_MMrotate_1x/'
dataset_type = 'ShipDataset'
default_hooks = dict(
    checkpoint=dict(interval=1, type='CheckpointHook'),
    logger=dict(interval=50, type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    timer=dict(type='IterTimerHook'),
    visualization=dict(type='mmdet.DetVisualizationHook'))
default_scope = 'mmrotate'
device = 'cuda:0'
env_cfg = dict(
    cudnn_benchmark=False,
    dist_cfg=dict(backend='nccl'),
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0))
gpu_ids = range(0, 1)
load_from = '/mnt/hdd/eric/.tmp_ipy/00.Weights/MTP/diorr-rvsa-l-mae-mtp-epoch_12.pth'
log_level = 'INFO'
log_processor = dict(by_epoch=True, type='LogProcessor', window_size=50)
model = dict(
  

In [8]:
cfg.pop('param_scheduler')

[{'type': 'LinearLR',
  'start_factor': 1e-06,
  'by_epoch': False,
  'begin': 0,
  'end': 500},
 {'type': 'MultiStepLR',
  'begin': 0,
  'end': 12,
  'by_epoch': True,
  'milestones': [8, 11],
  'gamma': 0.1}]

# Train a detector

In [9]:
# register all modules in mmdet into the registries
# do not init the default scope here because it will be init in the runner
register_all_modules_mmdet(init_default_scope=False)
register_all_modules(init_default_scope=False)

# register all modules in mmrotate into the registries
register_all_modules()

# build the model from a config file and a checkpoint file
model = init_detector(cfg, cfg.checkpoint, palette="dota", device=cfg.device)

runner = Runner.from_cfg(cfg)
runner.train()

/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
Loads checkpoint by local backend from path: /mnt/hdd/eric/.tmp_ipy/00.Weights/MTP/diorr-rvsa-l-mae-mtp-epoch_12.pth
The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([21, 1024]) from checkpoint, the shape 

/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/06.Gits/01.SAMRS_MTP/MTP/MMRotate_1_x/mmrotate/mmrotate/structures/bbox/rotated_boxes.py:192: UserWarning: The `clip` function does nothing in `RotatedBoxes`.
  warnings.warn('The `clip` function does nothing in `RotatedBoxes`.')
/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/06.Gits/01.SAMRS_MTP/MTP/MMRotate_1_x/mmrotate/mmrotate/structures/bbox/rotated_boxes.py:192: UserWarning: The `clip` function does nothing in `RotatedBoxes`.
  warnings.warn('The `clip` function does nothing in `RotatedBoxes`.')
/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/06.Gits/01.SAMRS_MTP/MTP/MMRotate_1_x/mmrotate/mmrotate/structures/bbox/rotated_boxes.py:192: UserWarning: The `clip` function does nothing in `RotatedBoxes`.
  warnings.warn('The `clip` function does nothing in `RotatedBoxes`.')
/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/06.Gits/01.SAMRS_MTP/MTP/MMRotate_1_x/mmrotate/mmrotate/structures/bbox/rotated_boxes.py:192: UserWarning: The `clip` function does nothing in `Rotate

04/24 17:09:03 - mmengine - INFO - Epoch(train)  [1][  50/1374]  base_lr: 1.0000e-03 lr: 1.0000e-03  eta: 5:59:05  time: 1.3107  data_time: 0.0204  memory: 10955  loss: 110.6609  loss_rpn_cls: 23.7442  loss_rpn_bbox: 12.8694  loss_cls: 62.1180  acc: 99.4141  loss_bbox: 11.9294
04/24 17:10:03 - mmengine - INFO - Epoch(train)  [1][ 100/1374]  base_lr: 1.0000e-03 lr: 1.0000e-03  eta: 5:42:04  time: 1.1941  data_time: 0.0038  memory: 10955  loss: 3.6792  loss_rpn_cls: 1.0536  loss_rpn_bbox: 0.4349  loss_cls: 1.8062  acc: 99.1211  loss_bbox: 0.3845
04/24 17:11:06 - mmengine - INFO - Epoch(train)  [1][ 150/1374]  base_lr: 1.0000e-03 lr: 1.0000e-03  eta: 5:40:51  time: 1.2505  data_time: 0.0037  memory: 10955  loss: 1.4514  loss_rpn_cls: 0.6450  loss_rpn_bbox: 0.1912  loss_cls: 0.5853  acc: 99.1211  loss_bbox: 0.0298
04/24 17:12:09 - mmengine - INFO - Epoch(train)  [1][ 200/1374]  base_lr: 1.0000e-03 lr: 1.0000e-03  eta: 5:41:01  time: 1.2696  data_time: 0.0036  memory: 10955  loss: 1.3361  l

In [ ]:
import os

cfg.dump("./ship_fine_tunning_v1.py")